In [1]:
%load_ext autoreload
%autoreload 2

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report
import torch
import sklearn
from WeaponData import WeaponData
from cnn import CnnNet
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils import adaptive_leraning_rate, weights_init
import shap
from pytorch_model_summary import summary

%config InlineBackend.figure_format = 'svg'



In [2]:
dataset = WeaponData()


['banknot', 'karta płatnicza', 'nóż', 'pistolet', 'portfel', 'smartphone']


In [3]:
net = CnnNet(100, len(dataset.classes),  c_kernels=[7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], in_channels=[
             3, 16, 32, 64, 86, 128, 128, 128, 128, 128, 128, 128, 128], out_channels=[16, 32, 64, 86, 128, 128, 128, 128, 128, 128, 128, 128, 128], padding_flag=True,
                maxpool_freq=4,
                activation_relu=True,
                fc_size=4,)
criterion = nn.NLLLoss()
net.apply(lambda m: weights_init(m, "xavier_uniform"))
lr = 0.0001
optimizer = torch.optim.AdamW(net.parameters(), lr=lr)
loss_test = []


ilość klas: 6
wielkość po warstawach conv: 12


In [4]:
if torch.cuda.is_available():
    net = net.cuda()
    net.cnn = net.cnn.cuda()
    print("GPU is available")
else:
    torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [5]:
def test(dataset, model, criterion):
    total = 0.
    predicted = 0.
    loss_test = 0.
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for data, labels in dataset.testloader:
            labels = labels.cuda()
            data = data.cuda()
            out = model(data)
            predicted_output = torch.argmax(out, dim=1)

            y_true = np.append(y_true, labels.cpu().detach().numpy(), 0)
            y_pred = np.append(y_pred, predicted_output.cpu().detach().numpy(), 0)
            total += labels.shape[0]
            predicted += torch.sum(predicted_output == labels).cpu().item()

            loss = criterion(out, labels)
            loss_test += loss.cpu().item()
    confusion_matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
    return (predicted/total*100), loss_test, confusion_matrix

In [6]:

old_loss = 0
loss_training = []
for e in range(15):
    old_param = net.parameters

    loss_temp = 0
    # pass through all data
    for data, exp in dataset.trainloader:
        exp = exp.cuda()
        # pass data to cuda
        data = data.cuda()
        # clear gradient from previous epoch
        optimizer.zero_grad()
        out = net(data)
        loss = criterion(out, exp)
        loss.backward()
        loss_temp += loss.item()
        optimizer.step()

    loss_training.append(loss_temp)
    acc, loss_test, mat = test(dataset, net, criterion)
    net, optimizer = adaptive_leraning_rate(loss_training[-1], optimizer, net, old_param, old_loss)
    old_loss = loss_training[-1]
    
    

    # ax= plt.subplot()
    # sns.heatmap(mat, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

    # ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    # ax.set_title('Confusion Matrix'); 
    # ax.xaxis.set_ticklabels(['1', '2', '3', '4', '5', '6']); ax.yaxis.set_ticklabels(['1', '2', '3', '4', '5', '6']);
    # plt.show()
    
    temp_lr: float = optimizer.param_groups[0]['lr']
    print("acc: {:.2f} %".format(acc))
    print("Learning rate: {:.10f}".format(temp_lr))
    print("Epoch: {}.............".format(e), end=" ")
    print("Loss: {:.4f}".format(loss))
    if acc > 80.0:
        break

torch.save(net, "./output_data/models/weapon_dataset")


acc: 49.00 %
Learning rate: 0.0001000000
Epoch: 0............. Loss: 1.3835
acc: 53.15 %
Learning rate: 0.0001040000
Epoch: 1............. Loss: 1.2892
acc: 66.47 %
Learning rate: 0.0001081600
Epoch: 2............. Loss: 0.7130
acc: 70.15 %
Learning rate: 0.0001124864
Epoch: 3............. Loss: 1.0657
acc: 70.51 %
Learning rate: 0.0001169859
Epoch: 4............. Loss: 0.7035
acc: 72.61 %
Learning rate: 0.0001216653
Epoch: 5............. Loss: 0.6936
acc: 72.82 %
Learning rate: 0.0001265319
Epoch: 6............. Loss: 0.9170
acc: 73.29 %
Learning rate: 0.0001315932
Epoch: 7............. Loss: 0.4689
acc: 75.08 %
Learning rate: 0.0001368569
Epoch: 8............. Loss: 0.5834
acc: 76.13 %
Learning rate: 0.0001423312
Epoch: 9............. Loss: 0.7187
acc: 75.81 %
Learning rate: 0.0001480244
Epoch: 10............. Loss: 0.4318
acc: 75.55 %
Learning rate: 0.0001539454
Epoch: 11............. Loss: 0.5484
acc: 74.71 %
Learning rate: 0.0001601032
Epoch: 12............. Loss: 0.2592
acc: 76.8